In [1]:
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorchtools
import glob as gl
import random
import os
import time
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_regression
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import log_loss
from numpy import hstack
from numpy import vstack

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()
find_directory(back_num = 1, foldername = ['Dataset'], filename = 'bat_dict.pkl')

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset\\Random seed test'

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset\\Dataset\\bat_dict.pkl'

In [3]:
bat_dict_add_1_2 = find_directory(back_num = 3, foldername = ['Dataset'], filename = 'bat_1_2_dict_VITQcQd.pkl')
with open(bat_dict_add_1_2, 'rb') as tf:
    bat_dict_1_2 = pickle.load(tf)

bat_sel_dict = bat_dict_1_2
max_time = 120

In [4]:
# Tensor dimension: (cell, cycle, time, variable)
num_cells = len(bat_sel_dict.keys())
regression_cycles = [30, 50, 80, 100]

# variable = [Qc, I, V, T, Qd]
num_variables = len(bat_sel_dict['b1c0']['1']['1.0'].keys())

for reg in regression_cycles:
    #'''RNN 가변길이 다룰수 있게?? 고민필요'''
    globals()[f'x_tensor_{reg}'] = torch.zeros(num_cells, reg, max_time, num_variables)
    cell_index = 1
    for cell in bat_sel_dict.keys():
        for cycle in range(1, reg+1):
            for time in bat_sel_dict[cell][str(cycle)].keys():
    #             print(bat_sel_dict[cell][str(cycle)][str(time)])
                value_list = [i for i in bat_sel_dict[cell][str(cycle)][str(time)].values()]
    #             print(value_list)
                globals()[f'x_tensor_{reg}'][cell_index-1, cycle-1, int(float(time)*max_time/60), :] = torch.FloatTensor(value_list)

        cell_index += 1

    globals()[f'x_tensor_{reg}'].size()

num_variables = globals()[f'x_tensor_{reg}'].shape[3]
time_length = globals()[f'x_tensor_{reg}'].shape[2]
# (cycle_index-1, cycle-1, time-1, variable(Qc, I, V, T, Qd))
# x_tensor[123, 99, 30, :]

torch.Size([124, 30, 120, 5])

torch.Size([124, 50, 120, 5])

torch.Size([124, 80, 120, 5])

torch.Size([124, 100, 120, 5])

In [5]:
y_df_add = find_directory(back_num = 3, foldername = ['Dataset', 'y_df.csv'])
y_df = pd.read_csv(y_df_add)

# 2. Regression problem
# KO, KP, CL
y_cl_numpy = y_df['Cycle life'].to_numpy()
y_kp_numpy = y_df['kneepoints'].to_numpy()
y_ko_numpy = y_df['kneeonsets'].to_numpy()
# y_ko_numpy.shape

y_regression_numpy = y_ko_numpy

y_ko_tensor = torch.FloatTensor(y_ko_numpy)
y_ko_tensor.size()

torch.Size([124])

In [6]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed) # torch 
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True # cudnn
    torch.backends.cudnn.benchmark = False # cudnn
    np.random.seed(random_seed) # numpy
    random.seed(random_seed) # random

In [7]:
globals()[f'x_tensor_{reg}'].shape

torch.Size([124, 100, 120, 5])

## Data loading

In [8]:
# y: no scaling
class dataPrep_RNN_CNN(Dataset):  
    def __init__(self, x_tensor, y_tensor, batch_size, scaler, test_split: float, random_seed = int):
        self.random_seed = random_seed
        self.xdata = torch.permute(x_tensor, (0, 3, 2, 1))
        self.ydata = y_tensor
        
        self.batch_size = batch_size
                
        self.xscaler = scaler()
        
        self.test_split = test_split
        
        self.train_size, self.val_size, self.test_size = self.get_splits()
        setRandomSeed(random_seed = self.random_seed)
        # Tensor dimension: (cell, variable, time, cycle)
        # If problem emerges, check this part first.
        for i in range(len(self.xdata)):
            for j in range(self.xdata.shape[1]):
                temp = np.expand_dims(self.xscaler.fit_transform(self.xdata[i,j,:,:]), axis=0)
                if j ==0:
                    temp2 = temp
                else:
                    temp2 = np.vstack((temp2,temp))
            temp2 = np.expand_dims(temp2, axis=0)
            if i==0:
                self.xdata_scaled = temp2
            else:
                self.xdata_scaled = np.vstack((self.xdata_scaled, temp2))

        self.ydata_scaled = self.ydata
        
        self.x_train_scaled, self.x_test_scaled, self.y_train_scaled, self.y_test_scaled = train_test_split(self.xdata_scaled, 
                                                                                                            self.ydata_scaled, 
                                                                                                            test_size = self.test_size, 
                                                                                                            shuffle = True, 
                                                                                                            random_state=self.random_seed)
        
        self.x_train_scaled, self.x_val_scaled, self.y_train_scaled, self.y_val_scaled = train_test_split(self.x_train_scaled,
                                                                                                         self.y_train_scaled,
                                                                                                         test_size = self.val_size, 
                                                                                                         shuffle = True, 
                                                                                                         random_state = self.random_seed)
        
        print(f"random seed = {self.random_seed},", self.x_train_scaled.shape, self.x_val_scaled.shape, self.x_test_scaled.shape,
              self.y_train_scaled.shape, self.y_val_scaled.shape, self.y_test_scaled.shape)
        
        self.train = list(zip(self.x_train_scaled, self.y_train_scaled))
        self.val = list(zip(self.x_val_scaled, self.y_val_scaled))
        self.test = list(zip(self.x_test_scaled, self.y_test_scaled))
                
        self.train_dataloader = DataLoader(self.train, batch_size = self.batch_size, shuffle=True)
        self.val_dataloader = DataLoader(self.val, batch_size = self.batch_size, shuffle=True)
        self.test_dataloader = DataLoader(self.test, batch_size = self.batch_size, shuffle=True)
            
    # split the datasets into training and testing
    def get_splits(self):
        # test size
        test_size = int(self.test_split*len(self.xdata))
        # val size
        val_size = int(0.2*(len(self.xdata) - test_size))
        # train_size
        train_size = len(self.xdata) - val_size - test_size
        return train_size, val_size, test_size
    
    # Returns dataloaders for training and validation datasets
    def train_data(self):
        return self.train_dataloader, self.val_dataloader
    
    # Returns dataloader for test datasets
    def test_data(self):
        return self.test_dataloader
    
    def scaler(self):
        return self.xscaler
    
    # finding length of x 
    def __len__(self):
        return len(self.X)
    
    # indexing rows for calling
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]


In [10]:
regression_cycles = [30, 50, 80, 100]
random_seeds = [0, 50, 150, 200, 100]

# x와 y가 1대1 대응이어야 dataloader를 만들 수 있어서 2D array 그대로 놓고 학습시 tensor transform 예정
for random_seed in random_seeds:
    for reg in regression_cycles:
        globals()[f'reg_dataset_{reg}_{random_seed}'] = dataPrep_RNN_CNN(globals()[f'x_tensor_{reg}'], y_ko_tensor, batch_size = 4, scaler = MinMaxScaler, test_split = 0.2, random_seed = random_seed)
        globals()[f'reg_train_dataloader_{reg}_{random_seed}'], globals()[f'reg_val_dataloader_{reg}_{random_seed}'] = globals()[f'reg_dataset_{reg}_{random_seed}'].train_data()
        globals()[f'reg_test_dataloader_{reg}_{random_seed}'] = globals()[f'reg_dataset_{reg}_{random_seed}'].test_data()
        globals()[f'reg_xscaler_{reg}_{random_seed}'] = globals()[f'reg_dataset_{reg}_{random_seed}'].scaler()

random seed = 0, (80, 5, 120, 30) (20, 5, 120, 30) (24, 5, 120, 30) torch.Size([80]) torch.Size([20]) torch.Size([24])
random seed = 0, (80, 5, 120, 50) (20, 5, 120, 50) (24, 5, 120, 50) torch.Size([80]) torch.Size([20]) torch.Size([24])
random seed = 0, (80, 5, 120, 80) (20, 5, 120, 80) (24, 5, 120, 80) torch.Size([80]) torch.Size([20]) torch.Size([24])
random seed = 0, (80, 5, 120, 100) (20, 5, 120, 100) (24, 5, 120, 100) torch.Size([80]) torch.Size([20]) torch.Size([24])
random seed = 50, (80, 5, 120, 30) (20, 5, 120, 30) (24, 5, 120, 30) torch.Size([80]) torch.Size([20]) torch.Size([24])
random seed = 50, (80, 5, 120, 50) (20, 5, 120, 50) (24, 5, 120, 50) torch.Size([80]) torch.Size([20]) torch.Size([24])
random seed = 50, (80, 5, 120, 80) (20, 5, 120, 80) (24, 5, 120, 80) torch.Size([80]) torch.Size([20]) torch.Size([24])
random seed = 50, (80, 5, 120, 100) (20, 5, 120, 100) (24, 5, 120, 100) torch.Size([80]) torch.Size([20]) torch.Size([24])
random seed = 150, (80, 5, 120, 30) (2

## Define model class

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [12]:
class RNNcell(nn.Module):
    def __init__(self, model, input_size, hidden_size, dropout = 0.1, bidirectional = False, num_layer=1):
        super(RNNcell, self).__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        self.model = model
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.D = 1 + self.bidirectional
        self.batch_size = 4
        
        #Batch size * cycles, timestep, input_size
        if self.model.endswith('RNN'):
            self.rnn = nn.RNN(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('LSTM'):
            self.rnn = nn.LSTM(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('GRU'):
            self.rnn = nn.GRU(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
            
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_uniform(param)
            else:
                nn.init.normal(param)
        
    def forward(self, x):
        # x made from (cell * n_cy, timestep(seq_len), num_vars)
        self.batch_size = x.size(0)
        
        D = 1 + self.bidirectional
        #out: Containing output features h_t from the last layer of LSTM for each t(if bidirectional ==True: concat of forward and backward hidden states)
        #h_n: Containing final hidden state in the sequence
        h0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
        if self.model.endswith('LSTM'):
            c0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
            # c_n: Containing final cell state in the sequence
            out, (hn, cn) = self.rnn(x, (h0, c0))
        else:
            out, hn = self.rnn(x, h0.detach())
        return out, hn

## RNN-TA-1D CNN

In [13]:
class RNN_TA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, fil2, pool2, npool2, fsize2, psize2, mids, random_seed, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        
        super(RNN_TA_1DCNN, self).__init__()
        
        self.random_seed = random_seed
        setRandomSeed(random_seed = self.random_seed)
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
        self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(hn1.size(1)/self.num_cycles)

        # Reshape for input
        # out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)
        # outs : (batch_size(num_cell), num_cycle, timestep, D1*hid1)
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        # ta: (batch_size, num_cycle, timestep, 1(D1*hid1->1))
        ta = self.sm1(self.lin1(outs1))
        # ta_outs: (batch_size, num_cycle, timestep, D1*hid1)=>(batch_size, D1*hid1, num_cycle, timestep)
        ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        # ct_vec: (batch_size, D1*hid1, num_cycle)
        ct_vec = torch.sum(ta_outs, -1)
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ta.squeeze()

## Self attention for CA

In [14]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(SelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads
        
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, inputs):
        # inputs shape: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = inputs.size()
        
        # Compute query, key, and value matrices
        query = self.query(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        key = self.key(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        value = self.value(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        
        # Compute scaled dot-product attention scores
        # attention_scores: [batch_size, num_heads, seq_len, seq_len]
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_size, dtype=torch.float32))
        attention_probs = self.softmax(attention_scores)
        
        # Apply attention to the value matrix and concatenate heads
        # context: [batch_size, num_heads, seq_len, head_size]=>[batch_size, seq_len, num_heads, head_size] 
        # => [batch_size, seq_len, num_heads * head_size]
        context = torch.matmul(attention_probs, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
#         context = nn.Linear(hidden_size, 1)(context)
        return attention_probs, context

## RNN-CA-1D CNN

In [15]:
class RNN_CA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, nhead2, fil2, pool2, npool2, fsize2, psize2, mids, random_seed, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_CA_1DCNN, self).__init__()
        
        self.random_seed = random_seed
        setRandomSeed(random_seed = self.random_seed)
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        # Pooling/Nonpooling layers
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # Params for self-attention(ca)
        self.nh2 = nhead2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
#         self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.ca = SelfAttention(self.D1*hid1, self.nh2)
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(hn1.size(1)/self.num_cycles)

        # Reshape for input
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        outs1 = torch.permute(outs1, (0, 3, 1, 2))
        ct_vec = outs1[:, :, :, -1].squeeze()
        ca, ct_vec = self.ca(ct_vec.transpose(2, 1))
#         print(f"After self attention: ca.size: {ca.size()}, ct_vec.size: {ct_vec.size()}")
        # ct_vec shape: [batch_size, D1*hid1, num_cycle]
        ct_vec = ct_vec.transpose(2, 1)
        
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ca

## RNN-TA-CA-1D CNN

In [16]:
class RNN_TA_CA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, nhead2, fil2, pool2, npool2, fsize2, psize2, mids, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_TA_CA_1DCNN, self).__init__()
        self.random_seed = random_seed
        setRandomSeed(random_seed = self.random_seed)
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        # Pooling/Nonpooling layers
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # Params for self-attention(ca)
        self.nh2 = nhead2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
        self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.ca = SelfAttention(self.D1*hid1, self.nh2)
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(hn1.size(1)/self.num_cycles)

        # Reshape for input
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        ta = self.sm1(self.lin1(outs1))
        ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        ct_vec = torch.sum(ta_outs, -1)
        ca, ct_vec = self.ca(ct_vec.transpose(2, 1))
        ct_vec = ct_vec.transpose(2, 1)
        
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ta.squeeze(), ca

In [17]:
# Function for saving and loading of training history
def save_data(D3_array, filename):
    with open(filename,"wb") as dat_:
        pickle.dump(D3_array,dat_)
        
def load_data(filename):
    with open(filename,"rb") as ld:
        x_temp = pickle.load(ld)
    return x_temp

# Early stopping class
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_epoch = 1
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, epoch, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.best_model = self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'At epoch {self.best_epoch}: Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
        return model

In [18]:
class train:
    def __init__(self, modelname, model: nn.Module, train_dataloader, val_dataloader, test_dataloader, epoch: int, learning_rate=0.01, patience = 5, random_seed = 100, verbose = False):
        super().__init__()
        #   Reprocudtion
        self.random_seed = random_seed
        self.modelname = modelname
        setRandomSeed(self.random_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(self.random_seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        os.environ['PYTHONHASHSEED'] = str(self.random_seed)
        torch.manual_seed(self.random_seed)
        np.random.seed(self.random_seed)

        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.test_dataloader = test_dataloader
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learning_rate)
        self.early_stopping = EarlyStopping(patience = patience, verbose = verbose)

        self.criterion = nn.MSELoss()
        self.name = ['train_rmse', 'val_rmse', 'test_rmse']
        
    def EvalModel(self):
        EvalPredictions, EvalActuals = list(), list()
        TestPredictions, TestActuals = list(), list()
        self.model.eval()
        with torch.no_grad():
            for n, (inputs, targets) in enumerate(self.val_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)
                
                if self.modelname == 'RNN_TA_1DCNN':
                    yhat, ta = self.model(inputs)
                elif self.modelname == 'RNN_CA_1DCNN':
                    yhat, ca = self.model(inputs)
                else:
                    yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                EvalPredictions.append(yhat)
                EvalActuals.append(actual)

            EvalPredictions, EvalActuals = vstack(EvalPredictions), vstack(EvalActuals)
            val_rmse = mean_squared_error(EvalActuals, EvalPredictions, squared = False)
            
            for n, (inputs, targets) in enumerate(self.test_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                if self.modelname == 'RNN_TA_1DCNN':
                    yhat, ta = self.model(inputs)
                elif self.modelname == 'RNN_CA_1DCNN':
                    yhat, ca = self.model(inputs)
                else:
                    yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                TestPredictions.append(yhat)
                TestActuals.append(actual)

            TestPredictions, TestActuals = vstack(TestPredictions), vstack(TestActuals)
            test_rmse = mean_squared_error(TestActuals, TestPredictions, squared = False)
            
            return val_rmse, test_rmse

    def TrainModel(self):
        loss_history = []
        self.model.train()
        for i in range(self.epoch):
            predictions, actuals = list(), list()
            for n, (inputs, targets) in enumerate(self.train_dataloader):
                self.model.train()
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                self.optimizer.zero_grad()

                if self.modelname == 'RNN_TA_1DCNN':
                    yhat, ta = self.model(inputs)
                elif self.modelname == 'RNN_CA_1DCNN':
                    yhat, ca = self.model(inputs)
                else:
                    yhat, ta, ca = self.model(inputs)
                    
                loss = self.criterion(yhat, targets)
                loss.backward()
                self.optimizer.step()
                
                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')
                    
                yhat = yhat.detach().numpy()
                predictions.append(yhat)
                
                targets = targets.numpy()
                actuals.append(targets)

            # evaluation of entire train data
            predictions, actuals = vstack(predictions), vstack(actuals)
            
            train_rmse = mean_squared_error(actuals, predictions, squared = False)

            # evaluation on validation data
            val_rmse, test_rmse = self.EvalModel()
            early_factor = val_rmse
            
            # training loss history
            loss_history.append([train_rmse, val_rmse, test_rmse])

            self.early_stopping(i+1, early_factor, self.model)
            
            if self.early_stopping.early_stop:
                print("Early stopping at best epoch: ", self.early_stopping.best_epoch)
                self.best_loss = loss_history[self.early_stopping.best_epoch-1]
                loss_history = loss_history[:self.early_stopping.best_epoch]
                
                if self.modelname == 'RNN_TA_1DCNN':
                    return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta
                elif self.modelname == 'RNN_CA_1DCNN':
                    return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ca
                else:
                    return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
            
        print(f"Ends at final epoch {self.epoch}")
        print(f"Best epoch: {self.early_stopping.best_epoch}")
        self.best_loss = loss_history[self.early_stopping.best_epoch-1]
        loss_history = loss_history[:self.early_stopping.best_epoch]
        
        if self.modelname == 'RNN_TA_1DCNN':
            return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta
        elif self.modelname == 'RNN_CA_1DCNN':
            return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ca
        else:
            return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
        
    def predict(self, data: torch.Tensor):
        return self.model(data)

## Implementation

In [19]:
def noheads_history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr, random_seed):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ta_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ta'],
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ta_add

def multihead_history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, random_seed):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ta_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ta'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    ca_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ca'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ta_add, ca_add

## Input reduction: RNN-TA-CA-1D CNN만 하면 되지않나?
### 이미 rs=100 결과 있음, random seed check
### Input reduction, multi-head는 결과 없음(80, 50, 30 cycle multihead 결과 필요)

In [20]:
# Hyperparameters
num_vars = globals()[f'x_tensor_{reg}'].size()[3]

modelnames = ['RNN_TA_CA_1DCNN']
# modelnames = ['RNN_CA_1DCNN', 'RNN_TA_CA_1DCNN']
# modelnames = ['RNN_TA_CA_1DCNN']
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
# num_cycless = [100, 30, 50, 80]
num_cycless = [30, 50, 80]
# random_seeds = [0, 50, 150, 200, 100]
random_seeds = [0, 50, 150, 200]
ep_pats = [[2000, 20], [3000, 30]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

# nh2s = [5, 1, 2, 3, 4]
nh2s = [5, 4, 3, 2]

import time

for random_seed in random_seeds:
    for modelname in modelnames:
        for [n_ep, patience] in ep_pats:
            for num_cycles in num_cycless:
                df = pd.DataFrame(columns = ['train_rmse', 'val_rmse', 'test_rmse'])
                print(f"random_seed = {random_seed}, modelname = {modelname}, num_cycle = {num_cycles}, epoch = {n_ep}, patience = {patience}")
                trdl = globals()[f'reg_train_dataloader_{num_cycles}_{random_seed}']
                vdl = globals()[f'reg_val_dataloader_{num_cycles}_{random_seed}']
                tedl = globals()[f'reg_test_dataloader_{num_cycles}_{random_seed}']
                for rnn1, hid1 in itertools.product(rnns, hids):
                    bi1 = False
                    for pool2, npool2 in itertools.product(pools, npools):
                        for fil2, lr in itertools.product(num_fils, lrs):
                            if modelname == 'RNN_TA_1DCNN':
                                history_add, state_dict_add, ta_add = noheads_history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                        rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr, random_seed)
                                if os.path.isfile(history_add):
                                    history = load_data(history_add)
                                    df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}'] = history.iloc[-1, :]
                                else:
                                    print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers', f"rnn = {rnn1}, hid = {hid1}, num_fil = {fil2}, lr = {lr}")
                                    start = time.time()
                                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, hid1, fil2, pool2, npool2, fsize2, psize2, mids, random_seed).to(device)
                                    model_train = train(modelname, model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False, random_seed = random_seed)

                                    best_model, best_loss, history, ta = model_train.TrainModel()

                                    print('best_loss = '+ str(best_loss))
                                    save_data(ta, ta_add)
                                    save_data(history, history_add)
                                    save_data(best_model.state_dict(), state_dict_add)

                                    print("time: ", time.time()-start)
                                    print("\n\n")
                                    df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}'] = history.iloc[-1, :]
                            else:
                                for nh2 in nh2s:
                                    
                                    all_hid1 = hid1 * nh2
                                    history_add, state_dict_add, ta_add, ca_add = multihead_history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                        rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, random_seed)
                                    if os.path.isfile(history_add):
                                        history = load_data(history_add)
                                        df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}_{nh2}'] = history.iloc[-1, :]
                                    else:
                                        print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers', f"rnn = {rnn1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                                        start = time.time()
                                        model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids, random_seed).to(device)

                                        # Train
                                        model_train = train(modelname, model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False, random_seed = random_seed)



                                        if modelname == 'RNN_CA_1DCNN':
                                            best_model, best_loss, history, ca = model_train.TrainModel()
                                        else:
                                            best_model, best_loss, history, ta, ca = model_train.TrainModel()
                                            save_data(ta, ta_add)

                                        print('best_loss = '+ str(best_loss))
                                        save_data(history, history_add)
                                        save_data(best_model.state_dict(), state_dict_add)
                                        save_data(ca, ca_add)

                                        print("time: ", time.time()-start)
                                        print("\n\n")
                                        df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}_{nh2}'] = history.iloc[-1, :]
                                        
                    dfadd = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', 'rstest_results', modelname], 
                        filename = f'{n_ep}_{patience}_rs_{random_seed}_split_1_2_{modelname}_all batches_{num_cycles} cycles.csv')
                    df.to_csv(dfadd)                                                                                                                                                                                       

random_seed = 0, modelname = RNN_TA_CA_1DCNN, num_cycle = 30, epoch = 2000, patience = 20
Pooling: 2 layers, Nonpooling: 1 layers rnn = GRU, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  37
best_loss = [155.2318, 158.65807, 250.5144]
time:  8.127463817596436



Pooling: 2 layers, Nonpooling: 1 layers rnn = GRU, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.001
Early stopping at best epoch:  53
best_loss = [164.21231, 162.7706, 196.43983]
time:  7.904488801956177



Pooling: 2 layers, Nonpooling: 1 layers rnn = GRU, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  53
best_loss = [117.348495, 162.0001, 210.13647]
time:  7.83285117149353



Pooling: 2 layers, Nonpooling: 1 layers rnn = GRU, hid per head = 7, num_heads = 4, num_fil = 5, lr = 0.01
Early stopping at best epoch:  19
best_loss = [147.65509, 161.00032, 186.16937]
time:  4.113527297973633



Pooling: 2 layers, Nonpooling: 1 layers rnn = GRU, h

Early stopping at best epoch:  13
best_loss = [167.84186, 171.94035, 246.64662]
time:  5.202788591384888



Pooling: 2 layers, Nonpooling: 2 layers rnn = GRU, hid per head = 7, num_heads = 4, num_fil = 5, lr = 0.01
Early stopping at best epoch:  42
best_loss = [152.08182, 152.54936, 226.14635]
time:  9.843724489212036



Pooling: 2 layers, Nonpooling: 2 layers rnn = GRU, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  37
best_loss = [122.085526, 154.48909, 213.14972]
time:  9.366427659988403



Pooling: 2 layers, Nonpooling: 2 layers rnn = GRU, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  37
best_loss = [137.56491, 115.102394, 202.38588]
time:  9.415074348449707



Pooling: 2 layers, Nonpooling: 2 layers rnn = GRU, hid per head = 7, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  4
best_loss = [535.99097, 551.2049, 630.53296]
time:  3.9995954036712646



Pooling: 2 layers, Nonpooling

Early stopping at best epoch:  53
best_loss = [161.20456, 170.56566, 209.02283]
time:  10.171674013137817



Pooling: 1 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  4
best_loss = [534.91864, 550.06494, 629.4212]
time:  3.4531056880950928



Pooling: 1 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.01
Early stopping at best epoch:  42
best_loss = [93.45623, 139.91429, 163.8978]
time:  8.843974828720093



Pooling: 1 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  4
best_loss = [534.92035, 550.06665, 629.42285]
time:  3.4062600135803223



Pooling: 1 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  62
best_loss = [91.31532, 155.15863, 154.5802]
time:  11.436877012252808



Pooling: 1 layers, Nonpooling: 1 la

Early stopping at best epoch:  102
best_loss = [79.35098, 131.75055, 134.34088]
time:  18.179960250854492



Pooling: 1 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  60
best_loss = [120.93104, 151.65283, 203.21057]
time:  11.944224834442139



Pooling: 1 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 3, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  161
best_loss = [210.82033, 202.80759, 308.87808]
time:  27.614328861236572



Pooling: 1 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 3, num_heads = 4, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  217
best_loss = [167.97318, 191.88924, 272.27634]
time:  35.48764491081238



Pooling: 1 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 3, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  220
best_loss = [235.15643, 236.99878, 300.95984]
time:  35.620646476745605



Pooling: 1 layers, 

Early stopping at best epoch:  4
best_loss = [534.7413, 549.8684, 629.21124]
time:  3.593045234680176



Pooling: 2 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 3, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  4
best_loss = [535.0714, 550.1765, 629.7317]
time:  3.5878865718841553



Pooling: 2 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 3, num_heads = 2, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  4
best_loss = [535.5265, 550.52826, 629.98413]
time:  3.587540626525879



Pooling: 2 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 3, num_heads = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  37
best_loss = [167.4668, 163.53928, 252.54419]
time:  8.809058427810669



Pooling: 2 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 3, num_heads = 4, num_fil = 7, lr = 0.001
Early stopping at best epoch:  61
best_loss = [155.54692, 162.47223, 263.31912]
time:  12.31294560432434



Pooling: 2 layers, Nonpooling: 1 la

...

Early stopping at best epoch:  4
best_loss = [535.0169, 550.05273, 629.5635]
time:  6.528944253921509



Pooling: 2 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  4
best_loss = [535.48364, 550.7189, 630.05566]
time:  6.518258094787598



Pooling: 2 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  4
best_loss = [536.2858, 551.5481, 630.8354]
time:  6.462355375289917



Pooling: 2 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  57
best_loss = [129.66132, 162.33847, 170.14087]
time:  16.5156569480896



Pooling: 2 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 5, num_heads = 4, num_fil = 7, lr = 0.001
Early stopping at best epoch:  42
best_loss = [161.77348, 184.42197, 218.40509]
time:  13.686759948730469



Pooling: 2 layers, Nonpooling: 2 la

Early stopping at best epoch:  37
best_loss = [134.23788, 150.84569, 218.9892]
time:  12.234688997268677



Pooling: 1 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.001
Early stopping at best epoch:  57
best_loss = [129.76468, 151.0821, 155.16653]
time:  15.515463590621948



Pooling: 1 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  37
best_loss = [166.98593, 160.6481, 220.98051]
time:  12.360645771026611



Pooling: 1 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.001
Early stopping at best epoch:  37
best_loss = [136.62784, 150.49751, 214.46]
time:  12.092235326766968



Pooling: 1 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 7, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  69
best_loss = [89.11298, 157.9682, 140.00201]
time:  17.924685955047607



Pooling: 1 layers, Nonpooling: 1

Early stopping at best epoch:  77
best_loss = [158.08905, 183.20134, 186.70137]
time:  20.09367322921753



Pooling: 1 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 7, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  85
best_loss = [73.22396, 149.34457, 122.57288]
time:  22.153375387191772



Pooling: 1 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.01
Early stopping at best epoch:  85
best_loss = [83.415276, 139.43391, 138.94125]
time:  21.743072032928467



Pooling: 1 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  40
best_loss = [84.155876, 148.67825, 157.92157]
time:  13.04682993888855



Pooling: 1 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.01
Early stopping at best epoch:  40
best_loss = [93.135, 143.05789, 162.1169]
time:  13.134706258773804



Pooling: 2 layers, Nonpooling: 1 l

Early stopping at best epoch:  161
best_loss = [70.58885, 150.76993, 130.79039]
time:  35.14897918701172



Pooling: 2 layers, Nonpooling: 1 layers rnn = RNN, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.01
Early stopping at best epoch:  40
best_loss = [115.78525, 149.14935, 194.6983]
time:  12.996148109436035



Pooling: 2 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  242
best_loss = [199.92073, 197.28464, 296.98242]
time:  51.68849301338196



Pooling: 2 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 7, num_heads = 4, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  155
best_loss = [213.33902, 237.1842, 297.36884]
time:  35.00009346008301



Pooling: 2 layers, Nonpooling: 2 layers rnn = RNN, hid per head = 7, num_heads = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  161
best_loss = [199.8414, 190.7771, 293.5209]
time:  37.673126459121704



Pooling: 2 layers, Nonpool

ValueError: dropout should be a number in range [0, 1] representing the probability of an element being zeroed

In [ ]:
# Hyperparameters
num_vars = globals()[f'x_tensor_{reg}'].size()[3]

modelnames = ['RNN_TA_1DCNN', 'RNN_CA_1DCNN', 'RNN_TA_CA_1DCNN']
# modelnames = ['RNN_CA_1DCNN', 'RNN_TA_CA_1DCNN']
# modelnames = ['RNN_TA_CA_1DCNN']
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycless = [100, 30, 50, 80]
random_seeds = [0, 50, 150, 200, 100]
ep_pats = [[2000, 20], [3000, 30]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [5, 1, 2, 3, 4]

import time

for random_seed in random_seeds:
    for modelname in modelnames:
        for [n_ep, patience] in ep_pats:
            for num_cycles in num_cycless:
                df = pd.DataFrame(columns = ['train_rmse', 'val_rmse', 'test_rmse'])
                print(f"random_seed = {random_seed}, modelname = {modelname}, num_cycle = {num_cycles}, epoch = {n_ep}, patience = {patience}")
                trdl = globals()[f'reg_train_dataloader_{num_cycles}_{random_seed}']
                vdl = globals()[f'reg_val_dataloader_{num_cycles}_{random_seed}']
                tedl = globals()[f'reg_test_dataloader_{num_cycles}_{random_seed}']
                for rnn1, hid1 in itertools.product(rnns, hids):
                    bi1 = False
                    for pool2, npool2 in itertools.product(pools, npools):
                        for fil2, lr in itertools.product(num_fils, lrs):
                            if modelname == 'RNN_TA_1DCNN':
                                history_add, state_dict_add, ta_add = noheads_history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                        rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr, random_seed)
                                if os.path.isfile(history_add):
                                    history = load_data(history_add)
                                    df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}'] = history.iloc[-1, :]
                                else:
                                    print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers', f"rnn = {rnn1}, hid = {hid1}, num_fil = {fil2}, lr = {lr}")
                                    start = time.time()
                                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, hid1, fil2, pool2, npool2, fsize2, psize2, mids, random_seed).to(device)
                                    model_train = train(modelname, model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False, random_seed = random_seed)

                                    best_model, best_loss, history, ta = model_train.TrainModel()

                                    print('best_loss = '+ str(best_loss))
                                    save_data(ta, ta_add)
                                    save_data(history, history_add)
                                    save_data(best_model.state_dict(), state_dict_add)

                                    print("time: ", time.time()-start)
                                    print("\n\n")
                            else:
                                for nh2 in nh2s:
                                    
                                    all_hid1 = hid1 * nh2
                                    history_add, state_dict_add, ta_add, ca_add = multihead_history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                        rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, random_seed)
                                    if os.path.isfile(history_add):
                                        history = load_data(history_add)
                                        df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}_{nh2}'] = history.iloc[-1, :]
                                    else:
                                        print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers', f"rnn = {rnn1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                                        start = time.time()
                                        model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids, random_seed).to(device)

                                        # Train
                                        model_train = train(modelname, model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False, random_seed = random_seed)



                                        if modelname == 'RNN_CA_1DCNN':
                                            best_model, best_loss, history, ca = model_train.TrainModel()
                                        else:
                                            best_model, best_loss, history, ta, ca = model_train.TrainModel()
                                            save_data(ta, ta_add)

                                        print('best_loss = '+ str(best_loss))
                                        save_data(history, history_add)
                                        save_data(best_model.state_dict(), state_dict_add)
                                        save_data(ca, ca_add)

                                        print("time: ", time.time()-start)
                                        print("\n\n")
                                        
                    dfadd = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', 'rstest_results', modelname], 
                        filename = f'{n_ep}_{patience}_rs_{random_seed}_split_1_2_{modelname}_all batches_{num_cycles} cycles.csv')
                    df.to_csv(dfadd)                                                                                                                                                                                       